In [1]:
from collections import Counter
from pyspark.sql import Row
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import HashingTF
from pyspark.ml.feature import IDF
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

dataPath = """File uploaded to /FileStore/tables/C000008.txt
File uploaded to /FileStore/tables/C000010.txt
File uploaded to /FileStore/tables/C000013.txt
File uploaded to /FileStore/tables/C000014.txt
File uploaded to /FileStore/tables/C000007.txt
File uploaded to /FileStore/tables/C000016.txt
File uploaded to /FileStore/tables/C000020.txt
File uploaded to /FileStore/tables/C000022.txt"""
dataPath = [i.split(' ')[-1] for i in dataPath.split('\n')]

In [2]:
columns = ["category", "text"]
df = sc.textFile(dataPath[0]) \
  .map(lambda x: x.split(',')) \
  .map(lambda x: (x[0], x[1])) \
  .toDF(columns)

In [3]:
df.select("category", 'text').take(2)

Out[62]: [Row(category='1', text='5月 09日 消息 深度 报告 权威 内参 证券 www.kl178.com 今日热点 五一 长假 过后 人民币 再度 逼近 8元 关口 8.0090 证券 市场 预期 美联储 将在 6月 暂停 导致 国际 市场上 美元 走势 疲软 高了 人民币 汇率 各方 预计 人民币 长期 持续 升值 不可避免 突破 8元 几成 定局 摩根 斯坦利 年底 人民币 升值 7.5元 此种 态势 预期 未来 市场 房地产板块 将在 人民币 升值 影响下 稳步 上扬 而对 外贸 依赖度 较高 的企业 则有 面临 业绩 下滑 危险 近期 8元 关口 心理 仍有 反复 投资者 不应 乐观 更多 详情 免费 咨询 021 64690729 登录 www.kl178.com 证券 资深 行业 研究员 提供 客观 深度 超前 投资 信息 作者 声明 机构 本人所 知情 范围内 机构 财产 利害 关系人 所述 文章内容 利害关系 本版 文章 纯属 个人观点 仅供参考 文责自负 读者 入市 风险 '),
 Row(category='1', text='5月 09日 消息 深度 报告 权威 内参 证券 www.kl178.com 今日热点 五一 长假 过后 人民币 再度 逼近 8元 关口 8.0090 证券 市场 预期 美联储 将在 6月 暂停 导致 国际 市场上 美元 走势 疲软 高了 人民币 汇率 各方 预计 人民币 长期 持续 升值 不可避免 突破 8元 几成 定局 摩根 斯坦利 年底 人民币 升值 7.5元 此种 态势 预期 未来 市场 房地产板块 将在 人民币 升值 影响下 稳步 上扬 而对 外贸 依赖度 较高 的企业 则有 面临 业绩 下滑 危险 近期 8元 关口 心理 仍有 反复 投资者 不应 乐观 更多 详情 免费 咨询 021 64690729 登录 www.kl178.com 证券 资深 行业 研究员 提供 客观 深度 超前 投资 信息 作者 声明 机构 本人所 知情 范围内 机构 财产 利害 关系人 所述 文章内容 利害关系 本版 文章 纯属 个人观点 仅供参考 文责自负 读者 入市 风险 ')]

In [4]:
tokenizer = Tokenizer().setInputCol('text').setOutputCol('word')
wordData = tokenizer.transform(df)

In [5]:
wordData.select("category","text","word").take(2)

Out[64]: [Row(category='1', text='5月 09日 消息 深度 报告 权威 内参 证券 www.kl178.com 今日热点 五一 长假 过后 人民币 再度 逼近 8元 关口 8.0090 证券 市场 预期 美联储 将在 6月 暂停 导致 国际 市场上 美元 走势 疲软 高了 人民币 汇率 各方 预计 人民币 长期 持续 升值 不可避免 突破 8元 几成 定局 摩根 斯坦利 年底 人民币 升值 7.5元 此种 态势 预期 未来 市场 房地产板块 将在 人民币 升值 影响下 稳步 上扬 而对 外贸 依赖度 较高 的企业 则有 面临 业绩 下滑 危险 近期 8元 关口 心理 仍有 反复 投资者 不应 乐观 更多 详情 免费 咨询 021 64690729 登录 www.kl178.com 证券 资深 行业 研究员 提供 客观 深度 超前 投资 信息 作者 声明 机构 本人所 知情 范围内 机构 财产 利害 关系人 所述 文章内容 利害关系 本版 文章 纯属 个人观点 仅供参考 文责自负 读者 入市 风险 ', word=['5月', '09日', '消息', '深度', '报告', '权威', '内参', '证券', 'www.kl178.com', '今日热点', '五一', '长假', '过后', '人民币', '再度', '逼近', '8元', '关口', '8.0090', '证券', '市场', '预期', '美联储', '将在', '6月', '暂停', '导致', '国际', '市场上', '美元', '走势', '疲软', '高了', '人民币', '汇率', '各方', '预计', '人民币', '长期', '持续', '升值', '不可避免', '突破', '8元', '几成', '定局', '摩根', '斯坦利', '年底', '人民币', '升值', '7.5元', '此种', '态势', '预期', '未来', '市场', '房地产板块', '将在', '人民币', '升值', '影响下', '稳步', '上扬', '而对', '外贸', '依赖度', '较高', '的企业', '则有', '面临', '业绩', '下滑', '危险', '近期', '8元', '关口', '心理', '仍有', '反复', '投资者', '不应', '乐观', '更多', '详情', '免费', '咨询', '021', '64690729', '登录', 'www.kl178.com', '证券', '资深', '行业', '研究员', '提供', '客观', '深度', '超前', '投资', '信息', '作者', '声明', '机构', '本人所', '知情', '范围内', '机构', '财产', '利害', '关系人', '所述', '文章内容', '利害关系', '本版', '文章', '纯属', '个人观点', '仅供参考', '文责自负', '读者', '入市', '风险']),
 Row(category='1', text='5月 09日 消息 深度 报告 权威 内参 证券 www.kl178.com 今日热点 五一 长假 过后 人民币 再度 逼近 8元 关口 8.0090 证券 市场 预期 美联储 将在 6月 暂停 导致 国际 市场上 美元 走势 疲软 高了 人民币 汇率 各方 预计 人民币 长期 持续 升值 不可避免 突破 8元 几成 定局 摩根 斯坦利 年底 人民币 升值 7.5元 此种 态势 预期 未来 市场 房地产板块 将在 人民币 升值 影响下 稳步 上扬 而对 外贸 依赖度 较高 的企业 则有 面临 业绩 下滑 危险 近期 8元 关口 心理 仍有 反复 投资者 不应 乐观 更多 详情 免费 咨询 021 64690729 登录 www.kl178.com 证券 资深 行业 研究员 提供 客观 深度 超前 投资 信息 作者 声明 机构 本人所 知情 范围内 机构 财产 利害 关系人 所述 文章内容 利害关系 本版 文章 纯属 个人观点 仅供参考 文责自负 读者 入市 风险 ', word=['5月', '09日', '消息', '深度', '报告', '权威', '内参', '证券', 'www.kl178.com', '今日热点', '五一', '长假', '过后', '人民币', '再度', '逼近', '8元', '关口', '8.0090', '证券', '市场', '预期', '美联储', '将在', '6月', '暂停', '导致', '国际', '市场上', '美元', '走势', '疲软', '高了', '人民币', '汇率', '各方', '预计', '人民币', '长期', '持续', '升值', '不可避免', '突破', '8元', '几成', '定局', '摩根', '斯坦利', '年底', '人民币', '升值', '7.5元', '此种', '态势', '预期', '未来', '市场', '房地产板块', '将在', '人民币', '升值', '影响下', '稳步', '上扬', '而对', '外贸', '依赖度', '较高', '的企业', '则有', '面临', '业绩', '下滑', '危险', '近期', '8元', '关口', '心理', '仍有', '反复', '投资者', '不应', '乐观', '更多', '详情', '免费', '咨询', '021', '64690729', '登录', 'www.kl178.com', '证券', '资深', '行业', '研究员', '提供', '客观', '深度', '超前', '投资', '信息', '作者', '声明', '机构', '本人所', '知情', '范围内', '机构', '财产', '利害', '关系人', '所述', '文章内容', '利害关系', '本版', '文章', '纯属', '个人观点', '仅供参考', '文责自负', '读者', '入市', '风险'])]

In [6]:
hashingTF = HashingTF().setInputCol("word").setOutputCol("feature").setNumFeatures(100)
featureData = hashingTF.transform(wordData)

In [7]:
featureData.show(2)

+--------+-----------------------------+--------------------------+--------------------+
category| text| word| feature|
+--------+-----------------------------+--------------------------+--------------------+
 1|5月 09日 消息 深度 报告 权...|[5月, 09日, 消息, 深度,...|(100,[0,2,4,5,7,8...|
 1|5月 09日 消息 深度 报告 权...|[5月, 09日, 消息, 深度,...|(100,[0,2,4,5,7,8...|
+--------+-----------------------------+--------------------------+--------------------+
only showing top 2 rows

In [8]:
idf = IDF().setInputCol('feature').setOutputCol('IDFfeature')
idfModel = idf.fit(featureData)
resData = idfModel.transform(featureData)

In [9]:
resData.show(2)

+--------+-----------------------------+--------------------------+--------------------+--------------------+
category| text| word| feature| IDFfeature|
+--------+-----------------------------+--------------------------+--------------------+--------------------+
 1|5月 09日 消息 深度 报告 权...|[5月, 09日, 消息, 深度,...|(100,[0,2,4,5,7,8...|(100,[0,2,4,5,7,8...|
 1|5月 09日 消息 深度 报告 权...|[5月, 09日, 消息, 深度,...|(100,[0,2,4,5,7,8...|(100,[0,2,4,5,7,8...|
+--------+-----------------------------+--------------------------+--------------------+--------------------+
only showing top 2 rows

In [10]:
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

training.show()

+---+----------------+-----+
 id| text|label|
+---+----------------+-----+
 0| a b c d e spark| 1.0|
 1| b d| 0.0|
 2| spark f g h| 1.0|
 3|hadoop mapreduce| 0.0|
+---+----------------+-----+

In [11]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.001)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])
model = pipeline.fit(training)

In [12]:
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])
prediction = model.transform(test)
selected = prediction.select("id", "text", "probability", "prediction")
for row in selected.collect():
    rid, text, prob, prediction = row
    print("(%d, %s) --> prob=%s, prediction=%f" % (rid, text, str(prob), prediction))

(4, spark i j k) --> prob=[0.1596407738787411,0.8403592261212589], prediction=1.000000
(5, l m n) --> prob=[0.8378325685476612,0.16216743145233878], prediction=0.000000
(6, spark hadoop spark) --> prob=[0.06926633132976266,0.9307336686702373], prediction=1.000000
(7, apache hadoop) --> prob=[0.9821575333444208,0.01784246665557917], prediction=0.000000

In [13]:
from pyspark.ml.feature import Word2Vec
documentDF = spark.createDataFrame([
    ("Hi I heard about Spark".split(" "), ),
    ("I wish Java could use case classes".split(" "), ),
    ("Logistic regression models are neat".split(" "), )
], ["text"])

word2Vec = Word2Vec(vectorSize=3, minCount=0, 
                    inputCol="text", outputCol="result")
model = word2Vec.fit(documentDF)
result = model.transform(documentDF)
for row in result.collect():
    text, vector = row
    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(vector)))

Text: [Hi, I, heard, about, Spark] => 
Vector: [-0.03504721522331238,0.008686093054711819,0.030792908370494844]

Text: [I, wish, Java, could, use, case, classes] => 
Vector: [0.02407229159559522,-0.047546425834298134,0.0704595111310482]

Text: [Logistic, regression, models, are, neat] => 
Vector: [-0.003552869707345963,0.06767420209944248,0.038231285661458975]

In [14]:
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.linalg import Vectors
df = spark.createDataFrame([
    (7, Vectors.dense([0.0, 0.0, 18.0, 1.0]), 1.0,),
    (8, Vectors.dense([0.0, 1.0, 12.0, 0.0]), 0.0,),
    (9, Vectors.dense([1.0, 0.0, 15.0, 0.1]), 0.0,)
], ["id", "features", "clicked"])

selector = ChiSqSelector(numTopFeatures=1, featuresCol="features",
                         outputCol="selectedFeatures", labelCol="clicked")
result = selector.fit(df).transform(df)
result.show()

+---+------------------+-------+----------------+
 id| features|clicked|selectedFeatures|
+---+------------------+-------+----------------+
 7|[0.0,0.0,18.0,1.0]| 1.0| [18.0]|
 8|[0.0,1.0,12.0,0.0]| 0.0| [12.0]|
 9|[1.0,0.0,15.0,0.1]| 0.0| [15.0]|
+---+------------------+-------+----------------+